In [1]:
# Import necessary libraries
import pickle
import os  
import pandas as pd
from fairness import * 
import numpy as np  
from sklearn.metrics import roc_curve, roc_auc_score 
import matplotlib.pyplot as plt  

# Try to load previously saved Data and sens_attr_dict objects from disk
try:
    # Load Data object containing pre-processed datasets
    with open('Data.pkl', 'rb') as f:
        Data = pickle.load(f)
    # Load sens_attr_dict containing sensitive attribute information
    with open('sens_attr_dict.pkl', 'rb') as f:
        sens_attr_dict = pickle.load(f)

# If loading fails (e.g., the files do not exist), proceed to manually process and generate these objects
except:
    Data = []  # Initialize an empty list to store processed data
    result = []  # Temporary list to store intermediate results

    # Iterate over files in the SCORES directory
    for row in os.listdir('SCORES'):
        # Skip system files
        if 'DS_Store' not in row:
            # Extract model and dataset names from the filename
            model, dataset = row.replace('score_', '').rstrip('.csv').split('_', 1)
            # For certain models, process the data differently
            if model not in ['SVM', 'LogReg', 'LinReg']:
                # Load true labels from the dataset
                y_true = np.array(pd.read_csv('DATA/' + dataset + '/test.csv')['label'])
                # Load the dataset into a DataFrame
                df = pd.read_csv('DATA/' + dataset + '/test.csv')
                # Load scores from the SCORES directory
                score = pd.read_csv('SCORES/' + row)
                score = np.array(score[score.columns[0]])
            else:
                # For specified datasets and models, handle differently
                if dataset == 'DBLP-GoogleScholar':
                    continue
                df_ = pd.read_csv('SCORES/' + row)
                y_true = np.array(df_['label']).reshape(-1)
                score = np.array(df_['score']).reshape(-1)
                # Process the DataFrame to rename columns based on sensitive attributes
                df_ = df_[['left', 'right']]
                df = df_.rename(columns={'left': 'left_' + sens_dict[dataset][0], 'right': 'right_' + sens_dict[dataset][0]})
            # Append the processed scores, true labels, model, dataset, and DataFrame to the result list
            result.append([score, y_true, model, dataset, df])

    sens_attr_dict = {}  # Initialize an empty dictionary for sensitive attributes

    # Iterate over the results to fill Data and sens_attr_dict
    for i, row in enumerate(result):
        dataset = row[-2]  # Extract dataset name
        df = row[-1]  # Extract DataFrame
        # If dataset is not in sens_attr_dict, create a sensitive attribute vector
        if dataset not in list(sens_attr_dict.keys()):
            sens_attr = make_sens_vector(df, dataset, sens_dict)
            sens_attr_dict[dataset] = sens_attr
        else:
            sens_attr = sens_attr_dict[dataset]
            # If the sensitive attribute vector does not match score shape, recreate it
            if sens_attr.shape[0] != score.shape[0]:
                sens_attr = make_sens_vector(df, dataset, sens_dict)
        # Append the score, true labels, sensitive attributes, model, and dataset to the Data list
        Data.append([row[0], row[1], sens_attr, row[2], row[3]])

    # Save the sens_attr_dict to disk for future use
    with open('sens_attr_dict.pkl', 'wb') as f:
        pickle.dump(sens_attr_dict, f)
    # Save the processed Data to disk for future use
    with open('Data.pkl', 'wb') as f:
        pickle.dump(Data, f)


## introduction figures

In [2]:
# Define constants for model and dataset to be analyzed
MODEL = 'LogReg'
DATASET = 'Amazon-Google'

# Filter Data for the specific model and dataset
for row in Data:
    [score, y_true, sens_attr, model, dataset] = row
    if dataset == DATASET and model == MODEL:
        # Partition scores based on sensitive attribute
        score_g1 = score[sens_attr == 1]
        score_g2 = score[sens_attr == 0]
        break

# Initialize lists to store fairness metrics over different thresholds
E_opp__sens = []
E_opp__non_sens = []
E_odds_sens = []
E_odds__non_sens = []

# Define range of thresholds to evaluate
start, end, step = 0, 1, 100
range = np.linspace(start, end, step)

# Calculate fairness metrics for each threshold
for TH in range:
    # Generate predictions based on threshold
    y_pred = np.array([1 if score > TH else 0 for score in score])
    # Calculate fairness metrics
    additional_fairness_metrics = calculate_additional_fairness_metrics2(y_true, y_pred, sens_attr)
    # Extract and store each metric
    E_opp__sens.append(additional_fairness_metrics[0]['e_opp__sens'])
    E_opp__non_sens.append(additional_fairness_metrics[0]['e_opp__non_sens'])
    E_odds_sens.append(additional_fairness_metrics[0]['e_odds_sens'])
    E_odds__non_sens.append(additional_fairness_metrics[0]['e_odds__non_sens'])

# Compute ROC curves and AUC scores for each group
fpr1, tpr1, _ = roc_curve(y_true[sens_attr == 1], score_g1, drop_intermediate=False)
fpr2, tpr2, _ = roc_curve(y_true[sens_attr == 0], score_g2, drop_intermediate=False)
auc1 = roc_auc_score(y_true[sens_attr == 1], score_g1)
auc2 = roc_auc_score(y_true[sens_attr == 0], score_g2)

# Calculate disparity in true positive rates
EO_opps_dist = calc_DP_TPR(sens_attr, y_true, score)

# Plot settings and color definitions
minority_col, majority_col, green_color = "#FF5733", "#33AFFF", "#C7E9B4"
L, F, F_legend, F_title, size = 1.5, 28, 22, 32, (8, 6)

# Plot Equal Opportunity metric
plt.figure(figsize=size)
plt.plot(range, E_opp__sens, label='Minority', color=minority_col)
plt.plot(range, E_opp__non_sens, label='Majority', color=majority_col)
plt.fill_between(range, E_opp__sens, E_opp__non_sens, color='#C0C0C0', alpha=0.3)
plt.xlabel('Threshold (' + r'$\tau$' + ')', fontsize=F_title)
plt.ylabel('TPR', fontsize=F_title)
plt.legend(fontsize=F_legend).get_frame().set_edgecolor('black')
plt.tight_layout()
plt.savefig('FIGURES/introduction/Intro_EQ_OPP.pdf')
plt.close()

# Plot ROC curve and fill areas under curves
plt.figure(figsize=size)
plt.plot(fpr1, tpr1, label='Minority', color=minority_col, linewidth=L)
plt.plot(fpr2, tpr2, label='Majority', color=majority_col, linewidth=L)
plt.fill_between(fpr1, tpr1, color=minority_col, alpha=0.2)
plt.fill_between(fpr2, tpr2, color=majority_col, alpha=0.2)
plt.plot([0, 1], [0, 1], color='black', linestyle='--')  # Diagonal line
plt.xlabel('FPR', fontsize=F_title)
plt.ylabel('TPR', fontsize=F_title)
plt.legend(fontsize=F_legend).get_frame().set_edgecolor('black')
plt.tight_layout()
plt.savefig('FIGURES/introduction/Intro_AUC.pdf')
plt.close()


## Initial Metrics

In [2]:
# Define color scheme for plotting and groupings for fairness metrics

color_dict = {'minority':"#FF5733",
              'majority': "#33AFFF",
              'total':'#006400'}
G_dict = {'ACC':['minority','majority'],
          'AUC':['minority','majority'],
          'Equalized opportunity':['majority','minority'],
          'Equalized odds':['minority','majority'],
          'F1-score':['minority','majority'],
          'Positive Rate':['majority','minority'],
          }
df =[]

# Iterate through each data point to calculate fairness and performance metrics
for row in Data:
    [score, y_true,sens_attr ,model,dataset] = row
    # Split scores by sensitive attribute
    score_g1 = score[sens_attr ==1]
    score_g2 = score[sens_attr ==0]

    # Calculate AUC for both groups and overall
    auc_g1 = roc_auc_score(y_true[sens_attr==1], score[sens_attr ==1])
    auc_g2 = roc_auc_score(y_true[sens_attr==0], score[sens_attr ==0])
    auc = roc_auc_score(y_true, score)
    
    
    # Calculate fairness disparity metrics
    Eodd_disp = calc_EO_disp(sens_attr, y_true, score)
    Eop_disp = calc_DP_TPR(sens_attr, y_true, score)
    PR_disp = calc_DP_PR(sens_attr, y_true, score)

    # Predict and calculate metrics for threshold = 0.5
    y_pred = np.array([1 if score > 0.5 else 0 for score in score])
    
    # F1 score, accuracy, and positive rate for minority and majority, and overall
    f1_g1_5 = f1_score(y_true[sens_attr ==1], y_pred[sens_attr ==1])
    f1_g2_5 = f1_score(y_true[sens_attr ==0], y_pred[sens_attr ==0])
    f1_5 = f1_score(y_true, y_pred)
    
    # Calculate additional fairness metrics at threshold = 0.5
    accuracy_g1_5 = accuracy_score(y_true[sens_attr ==1], y_pred[sens_attr ==1])
    accuracy_g2_5 = accuracy_score(y_true[sens_attr ==0], y_pred[sens_attr ==0])
    accuracy_5 = accuracy_score(y_true, y_pred)

    # Predict and calculate metrics for threshold = 0.9, following similar steps as for threshold = 0.5
    tn, fp, fn, tp = confusion_matrix(y_true[sens_attr ==1], y_pred[sens_attr ==1]).ravel()
    PR_g1_5 = (tp + fp) / (tp+tn+fp+fn)
    tn, fp, fn, tp = confusion_matrix(y_true[sens_attr ==0], y_pred[sens_attr ==0]).ravel()
    PR_g2_5 = (tp + fp) / (tp+tn+fp+fn)

    additional_fairness_metrics = calculate_additional_fairness_metrics2(y_true, y_pred, sens_attr)[0]
    E_op_g1 = (additional_fairness_metrics['e_opp__sens'])
    E_op_g2 = (additional_fairness_metrics['e_opp__non_sens'] )
    E_od_g1 = (additional_fairness_metrics['e_odds_sens'])
    E_od_g2 = (additional_fairness_metrics['e_odds__non_sens'])


    E_op_5 = np.abs(E_op_g1 - E_op_g2)
    E_od_5 = np.abs(E_od_g2 - E_od_g1)
    PR_5 = np.abs(PR_g1_5 - PR_g2_5)
    delta_auc = np.abs(auc_g1 - auc_g2)


################################

    y_pred = np.array([1 if score > 0.9 else 0 for score in score])

    f1_g1_9 = f1_score(y_true[sens_attr ==1], y_pred[sens_attr ==1])
    f1_g2_9 = f1_score(y_true[sens_attr ==0], y_pred[sens_attr ==0])
    f1_9 = f1_score(y_true, y_pred)

    accuracy_g1_9 = accuracy_score(y_true[sens_attr ==1], y_pred[sens_attr ==1])
    accuracy_g2_9 = accuracy_score(y_true[sens_attr ==0], y_pred[sens_attr ==0])
    accuracy_9 = accuracy_score(y_true, y_pred)

    tn, fp, fn, tp = confusion_matrix(y_true[sens_attr ==1], y_pred[sens_attr ==1]).ravel()
    PR_g1_9 = (tp + fp) / (tp+tn+fp+fn)
    tn, fp, fn, tp = confusion_matrix(y_true[sens_attr ==0], y_pred[sens_attr ==0]).ravel()
    PR_g2_9 = (tp + fp) / (tp+tn+fp+fn)

    additional_fairness_metrics = calculate_additional_fairness_metrics2(y_true, y_pred, sens_attr)[0]
    E_op_g1 = (additional_fairness_metrics['e_opp__sens'])
    E_op_g2 = (additional_fairness_metrics['e_opp__non_sens'] )
    E_od_g1 = (additional_fairness_metrics['e_odds_sens'])
    E_od_g2 = (additional_fairness_metrics['e_odds__non_sens'])



    E_op_9 = np.abs(E_op_g1 - E_op_g2)
    E_od_9 = np.abs(E_od_g2 - E_od_g1)
    PR_9 = np.abs(PR_g1_9 - PR_g2_9)











    METRICS_dict = {
    'Dataset': dataset,
    'Model': model,
    'Distributioanl disparity: Equal opportunity (TPR)': Eop_disp ,
    'Distributioanl disparity: Equalized odds': Eodd_disp,
    'Distributioanl disparity: PR': PR_disp,

    'Positive Rate Parity (Threshold = 0.5)': PR_5,
    'Equalized odds Parity (Threshold = 0.5)': E_od_5,
    'Equal opportunity Parity (Threshold = 0.5)': E_op_5,
    
    'Total Accuracy (Threshold = 0.5)': accuracy_5,
    'Minority Accuracy (Threshold = 0.5)': accuracy_g1_5,
    'Majority Accuracy (Threshold = 0.5)': accuracy_g2_5,

    'Total F1 (Threshold = 0.5)': f1_5,
    'Minority F1 (Threshold = 0.5)': f1_g1_5,
    'Majority F1 (Threshold = 0.5)': f1_g2_5,


    'Positive Rate Parity (Threshold = 0.9)': PR_9,
    'Equalized odds Parity (Threshold = 0.9)': E_od_9,
    'Equal opportunity Parity (Threshold = 0.9)': E_op_9,
    
    'Total Accuracy (Threshold = 0.9)': accuracy_9,
    'Minority Accuracy (Threshold = 0.9)': accuracy_g1_9,
    'Majority Accuracy (Threshold = 0.9)': accuracy_g2_9,

    'Total F1 (Threshold = 0.9)': f1_9,
    'Minority F1 (Threshold = 0.9)': f1_g1_9,
    'Majority F1 (Threshold = 0.9)': f1_g2_9,


    'Total AUC': auc,
    'Minority AUC': auc_g1,
    'Majority AUC': auc_g2,
    'Delta AUC': np.abs(auc_g1 - auc_g2),



    }

    df_new = pd.DataFrame(METRICS_dict, index=[0])


    try:
        df = pd.concat([df, df_new], ignore_index=True)
    except:
        df = copy.deepcopy(df_new)



# Save the compiled metrics to a CSV file
df.to_csv('Metric_initial.csv',index = False)


## Before/After calibration Figures

In [ ]:
# Define color codes for different groups
color_dict = {'minority':"#FF5733",
              'majority': "#33AFFF",
              'total':'#006400'}

# Define groups for different metrics
G_dict = {'ACC':['minority','majority'],
          'AUC':['minority','majority'],
          'Equalized opportunity':['majority','minority'],
          'Equalized odds':['minority','majority'],
          'F1-score':['minority','majority'],
          'Positive Rate':['majority','minority'],
          }

# Initialize counters and data lists
cnt = 0 
df1 = []
df2 = []
df3 = []
df4 = []

# Loop through the data
for row in Data:
    # Unpack data from row
    [score, y_true,sens_attr ,model,dataset] = row
    score_g1 = score[sens_attr ==1]  # Scores for sensitive attribute group 1
    score_g2 = score[sens_attr ==0]  # Scores for sensitive attribute group 2
    
    # Calculate barycenter Wasserstein distance
    bary_wass, A, bin_centers1, bin_centers2 = calc_bary2(score,sens_attr, True)
    
    # Determine the number of bins for histograms
    num  = min(int(max(calc_bin(score_g1), calc_bin(score_g2))), 400)
    hist1, bin_edges1 = np.histogram(score_g1, bins=np.linspace(0, 1, num+1 ))
    hist2, bin_edges2 = np.histogram(score_g2, bins=np.linspace(0, 1, num+1 ))
    bin_centers1_ = 0.5 * (bin_edges1[:-1] + bin_edges1[1:])
    bin_centers2_ = 0.5 * (bin_edges2[:-1] + bin_edges2[1:])
    hist1 = hist1 / np.sum(hist1)
    hist2 = hist2 / np.sum(hist2)

    # Define and fit mapping transport for histograms
    mapper1 = ot.da.MappingTransport(mu=1e-3, eta=1e-20, bias=False, max_iter=2000, verbose= False, metric = 'euclidean', tol = 1e-5)
    mapper1.fit(Xs=hist1.reshape(-1, 1),Xt = bary_wass.reshape(-1,1))
    mapper2 = ot.da.MappingTransport(mu=1e-3, eta=1e-20, bias=False, max_iter=2000, verbose= False, metric = 'euclidean',tol = 1e-5)
    mapper2.fit(Xs=hist2.reshape(-1, 1), Xt =  bary_wass.reshape(-1,1))

    # Transform histograms based on mappings
    scores_list_1_mapped = mapper1.transform(Xs=hist1.reshape(-1, 1)).ravel()
    scores_list_2_mapped = mapper2.transform(Xs=hist2.reshape(-1, 1)).ravel()

    # Apply optimal transport to adjust score distributions
    original_hist, _ = np.histogram(score_g1, bins=len(scores_list_1_mapped), range=(min(bin_edges1), max(bin_edges1)))
    original_hist = original_hist / np.sum(original_hist)
    target_hist = scores_list_1_mapped / np.sum(scores_list_1_mapped)
    original_bin_midpoints = (np.linspace(min(bin_edges1), max(bin_edges1), len(original_hist))[:-1] + np.linspace(min(bin_edges1), max(bin_edges1), len(original_hist))[1:]) / 2
    target_bin_midpoints = (bin_edges1[:-1] + bin_edges1[1:]) / 2
    cost_matrix = ot.dist(bin_centers1[:,None], target_bin_midpoints[:, None], metric='sqeuclidean')
    optimal_transport_plan = ot.emd(original_hist, target_hist, cost_matrix)
    transformed_indices = np.argmax(optimal_transport_plan, axis=1)
    transformed_data = np.interp(score_g1, bin_centers1, target_bin_midpoints[transformed_indices])

    original_hist, _ = np.histogram(score_g2, bins=len(scores_list_2_mapped), range=(min(bin_edges1), max(bin_edges1)))
    original_hist = original_hist / np.sum(original_hist)
    target_hist = scores_list_2_mapped / np.sum(scores_list_2_mapped)
    original_bin_midpoints = (np.linspace(min(bin_edges1), max(bin_edges1), len(original_hist))[:-1] + np.linspace(min(bin_edges1), max(bin_edges1), len(original_hist))[1:]) / 2
    target_bin_midpoints = (bin_edges1[:-1] + bin_edges1[1:]) / 2
    cost_matrix = ot.dist(bin_centers1[:,None], target_bin_midpoints[:, None], metric='sqeuclidean')
    optimal_transport_plan = ot.emd(original_hist, target_hist, cost_matrix)
    transformed_indices = np.argmax(optimal_transport_plan, axis=1)
    transformed_data2 = np.interp(score_g2, bin_centers1, target_bin_midpoints[transformed_indices])

    map_score = np.zeros(score.shape)
    map_score[sens_attr == 1] = transformed_data 
    map_score[sens_attr == 0] = transformed_data2

    # Define objective function for gamma optimization
    def objective(gamma, score, y_true, sens_attr,score_repair):
        score_best = score * (1-gamma) + gamma * score_repair
        Eodd_disp = calc_EO_disp(sens_attr, y_true, score_best)
        Eop_disp = calc_DP_TPR(sens_attr, y_true, score_best)
        PR_disp = calc_DP_PR(sens_attr, y_true, score_best)
        return Eodd_disp, Eop_disp, PR_disp

    # Initialize lists to store objective function values
    func_Eodd,func_PR,func_Eop = [], [] ,[]
    
    # Calculate objective functions for different gamma values
    for gamma in np.linspace(0, 1, 200):
        Eodd_disp, Eop_disp, PR_disp = objective(gamma, score, y_true, sens_attr, score_repair= map_score)
        func_Eodd.append([gamma, Eodd_disp ])
        func_Eop.append([gamma, Eop_disp])
        func_PR.append([gamma, PR_disp])

    # Sort the objective functions and select the optimal gamma values
    func_Eop.sort(key= lambda x:x[1])
    func_Eodd.sort(key= lambda x:x[1])
    func_PR.sort(key= lambda x:x[1])

    # Increment the counter and print progress
    cnt+=1
    print(cnt,'/',len(Data),':',MODEL+ ' '+ dataset)

    # Extract the optimal gamma values
    gamma_Eop = func_Eop[0][0]
    gamma_Eodd = func_Eodd[0][0]
    gamma_PR = func_PR[0][0]
    
    # Apply the optimal gamma values to obtain optimal scores
    score_optimal_Eop = score * (1-gamma_Eop) + gamma_Eop * map_score
    score_optimal_Eodd = score * (1-gamma_Eodd) + gamma_Eodd * map_score
    score_optimal_PR = score * (1-gamma_PR) + gamma_PR * map_score

    # Plot the before and after distributions
    plot_bef_after(score_optimal_Eop,score_optimal_Eodd,score_optimal_PR, model,dataset,sens_attr, y_true, score)

    # Save the metrics to dataframes
    df1 = do_job(df1 , score_optimal_Eop,sens_attr, y_true, dataset, model, G_dict, color_dict, F, F_title, L , size , F_legend, 'final_Eop',gamma_Eop)
    df2 = do_job(df2 , score_optimal_Eodd,sens_attr, y_true, dataset, model, G_dict, color_dict, F, F_title, L , size , F_legend, 'final_Eodd',gamma_Eodd)
    df3 = do_job(df3 , score_optimal_PR,sens_attr, y_true, dataset, model, G_dict, color_dict, F, F_title, L , size , F_legend, 'final_PR',gamma_PR)

# Save the dataframes to CSV files
df1.to_csv('Metric_optimal_Eop.csv',index = False)
df2.to_csv('Metric_optimal_Eodd.csv',index = False)
df3.to_csv('Metric_optimal_PR.csv',index = False)
